# ベキ乗法

絶対値が最大の固有値とその対応する固有ベクトルを求めるための反復的なアルゴリズム

引数

* A: 固有値・固有ベクトルを求めたい行列（入力データ）。
* u_initial: 反復計算の開始点となる初期推定ベクトル。
* k_max: 最大反復回数（計算の制限）。
* epsilon: 収束判定に使用する許容誤差。

変数

* u_k: 現在の反復ステップでのベクトル $u^{(k)}$ を格納します。ループ内で更新されます。
* y_k: 現在のステップで正規化されたベクトル $y^{(k)}$ を格納します。
* lambda_k: 現在のステップで計算された固有値の推定値 $\lambda^{(k)}$ を格納します。
* k: for k in range(k_max) で使用される反復回数（ループカウンタ）。

In [3]:
import numpy as np

def power_method_improved(A, u_initial, k_max, epsilon):

    # 初期設定
    u_k = u_initial.copy()
    lambda_k = 0.0 # 固有値の初期値

    print(f"--- 反復計算開始 ---")

    # 反復処理
    for k in range(k_max):
        print(f"\n===== k={k} (反復回数) =====")

        # 1. ベクトルの正規化: y^(k) = u^(k) / ||u^(k)||_2
        # k=0 のとき、y^(1) が計算される
        norm_u_k = np.linalg.norm(u_k, 2)
        if norm_u_k == 0:
            raise ValueError("ノルムがゼロになりました。初期ベクトルを見直してください。")

        y_k = u_k / norm_u_k
        print(f"y^({k+1}) の値 (正規化ベクトル): {y_k}")

        # 2. 次の反復ベクトルを計算: u^(k+1) = A y^(k)
        u_k_plus_1 = A @ y_k

        # 3. 固有値の推定値 λ^(k+1) を計算: λ^(k+1) = (y^(k), u^(k+1))
        # これはレイリー商の推定値です。
        lambda_k = np.dot(y_k, u_k_plus_1)

        # 4. 収束判定: ||u^(k+1) - λ^(k+1)y^(k)||_2 <= ε なら break
        residual_norm = np.linalg.norm(u_k_plus_1 - lambda_k * y_k, 2)

        print(f"近似固有値 λ^({k+1}) [レイリー商]: {lambda_k:.6f}")
        print(f"残差ノルム ||A y^({k+1}) - λ^({k+1}) y^({k+1})||: {residual_norm:.6e}")
        print(f"次の反復ベクトル u^({k+2}) のノルム: {np.linalg.norm(u_k_plus_1):.6f}")

        if residual_norm <= epsilon:
            print(f"\n✅ 収束しました（反復回数: {k+1} 回）。")
            break

        # 次の反復のために u^(k) を更新
        u_k = u_k_plus_1

    else:
        # k_max に達しても収束しなかった場合
        print(f"\n⚠️ 警告: 最大反復回数 k_max={k_max} に達しましたが、収束しませんでした。")

    # 最終的な固有値と固有ベクトルを代入
    lambda_val = lambda_k
    # 最後の y^(k) が固有ベクトル x の近似値
    # k_max=0 の場合でもエラーにならないように 'y_k' の存在を確認
    x = y_k if 'y_k' in locals() else u_initial / np.linalg.norm(u_initial, 2)

    return lambda_val, x

# --- 実行例 ---
if __name__ == '__main__':
    # 例として、対象となる行列 A を定義
    A = np.array([
        [1, 1],
        [1, 0]
    ])

    # パラメータの設定
    u_0 = np.array([1.0, 0.0])  # 初期ベクトル u^(0)
    K_MAX = 50                 # 最大反復回数
    EPSILON = 1e-6             # 許容誤差 ε

    print(f"--- ベキ乗法の実行 ---")
    print(f"対象行列 A:\n{A}")
    print(f"初期ベクトル u^(0): {u_0}")
    print(f"最大反復回数 k_max: {K_MAX}")
    print(f"許容誤差 ε: {EPSILON}")

    try:
        # ベキ乗法の実行
        eigenvalue, eigenvector = power_method_improved(A, u_0, K_MAX, EPSILON)

        print("\n==============================")
        print("✅ 最終結果")
        print("==============================")
        print(f"最大固有値 λ の近似値: {eigenvalue:.6f}")
        print(f"対応する固有ベクトル x の近似値:\n{eigenvector}")

        # 検証 (Ax ≈ λx の確認)
        Ax = A @ eigenvector
        lambda_x = eigenvalue * eigenvector
        verification_diff = np.linalg.norm(Ax - lambda_x)
        print(f"\n検証 (||Ax - λx|| のノルム): {verification_diff:.6e}")

    except ValueError as e:
        print(f"\nエラーが発生しました: {e}")

--- ベキ乗法の実行 ---
対象行列 A:
[[1 1]
 [1 0]]
初期ベクトル u^(0): [1. 0.]
最大反復回数 k_max: 50
許容誤差 ε: 1e-06
--- 反復計算開始 ---

===== k=0 (反復回数) =====
y^(1) の値 (正規化ベクトル): [1. 0.]
近似固有値 λ^(1) [レイリー商]: 1.000000
残差ノルム ||A y^(1) - λ^(1) y^(1)||: 1.000000e+00
次の反復ベクトル u^(2) のノルム: 1.414214

===== k=1 (反復回数) =====
y^(2) の値 (正規化ベクトル): [0.70710678 0.70710678]
近似固有値 λ^(2) [レイリー商]: 1.500000
残差ノルム ||A y^(2) - λ^(2) y^(2)||: 5.000000e-01
次の反復ベクトル u^(3) のノルム: 1.581139

===== k=2 (反復回数) =====
y^(3) の値 (正規化ベクトル): [0.89442719 0.4472136 ]
近似固有値 λ^(3) [レイリー商]: 1.600000
残差ノルム ||A y^(3) - λ^(3) y^(3)||: 2.000000e-01
次の反復ベクトル u^(4) のノルム: 1.612452

===== k=3 (反復回数) =====
y^(4) の値 (正規化ベクトル): [0.83205029 0.5547002 ]
近似固有値 λ^(4) [レイリー商]: 1.615385
残差ノルム ||A y^(4) - λ^(4) y^(4)||: 7.692308e-02
次の反復ベクトル u^(5) のノルム: 1.617215

===== k=4 (反復回数) =====
y^(5) の値 (正規化ベクトル): [0.85749293 0.51449576]
近似固有値 λ^(5) [レイリー商]: 1.617647
残差ノルム ||A y^(5) - λ^(5) y^(5)||: 2.941176e-02
次の反復ベクトル u^(6) のノルム: 1.617914

===== k=5 (反復回数) =====
y^(6) の値 (正規化ベク